In [1]:
import pycqed as pq
import numpy as np
from pycqed.measurement import measurement_control
from pycqed.measurement.sweep_functions import None_Sweep
import pycqed.measurement.detector_functions as det
from qcodes import station
station = station.Station()


Data directory set to: /Users/Adriaan/Documents/Testing/Data


<IPython.core.display.Javascript object>

## Creating an instance of the measurement control

In [2]:

MC = measurement_control.MeasurementControl('MC',live_plot_enabled=True, verbose=False)
MC.station = station
station.add_component(MC)

'MC'

# A simple 1D measurement 

In [3]:


None_Sweep()
MC.set_sweep_function(None_Sweep())
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Soft())
dat = MC.run('dummy')

# A simple 2D measurement

In [4]:
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 5)
MC.set_sweep_function(None_Sweep(sweep_control='soft'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Soft())
dat=MC.run('test', mode='2D')

/Users/Adriaan/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/pycqed/measurement/measurement_control.py:499: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.TwoD_array[y_ind, x_ind, j] = self.dset[i, z_ind]
/Users/Adriaan/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/pycqed/measurement/measurement_control.py:499: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.TwoD_array[y_ind, x_ind, j] = self.dset[i, z_ind]
/Users/Adriaan/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/pycqed/measurement/measurement_control.py:499: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.TwoD_array[y_ind, x_ind, j] = self.dset[i, z_ind]


# 2D combinatioin of a hard inner and soft outer loop
The hard inner loop returns 30 values 

In [5]:
from importlib import reload
reload(det)

<module 'pycqed.measurement.detector_functions' from '/Users/Adriaan/GitHubRepos/DiCarloLab_Repositories/PycQED_py3/pycqed/measurement/detector_functions.py'>

In [6]:
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 30)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Hard(delay=.05, noise=.2))
dat = MC.run('2D_hard', mode='2D')

start idx: 0
start idx: 30
start idx: 60
start idx: 90
start idx: 120
start idx: 150
start idx: 180
start idx: 210
start idx: 240
start idx: 270
start idx: 300
start idx: 330
start idx: 360
start idx: 390
start idx: 420
start idx: 450
start idx: 480
start idx: 510
start idx: 540
start idx: 570
start idx: 600
start idx: 630
start idx: 660
start idx: 690
start idx: 720
start idx: 750
start idx: 780
start idx: 810
start idx: 840
start idx: 870


## A Hard measurement that uses soft averaging

The number of soft_averages determines how many times the experiment will be performed. 
Only the averaged data is plotted and saved. 
The number of soft-averages can be set as a parameter of the Measurement Control. 

Will first implement it for 1D hard sweeps (easier) and then follow for combinations of hard and soft sweeps. 

In [7]:
MC.soft_avg(10)

In [8]:

MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=.1))
dat = MC.run('dummy_hard')

start idx: 0
start idx: 0


TypeError: Can't broadcast (30, 2) -> (60, 2)

array([ 0.65631164,  0.03746736,  0.74947848,  0.27202054,  0.42844457,
        0.3059217 ,  0.58674701,  0.25134404,  0.67448748,  0.73906571,
        0.18573425,  0.46660944,  0.1131018 ,  0.32144482,  0.32103409,
        0.25989586,  0.95510552,  0.18853773,  0.29510219,  0.47458454])